In [1]:
import os
import re
import csv
import sqlite3
from sqlite3 import Error
import pandas as pd
#from pandas_datareader import data
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline

Connecting to SQLite

In [2]:
def create_connection(db_file, delete_db=False):
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

SQLite Queries

In [146]:
# Create Table Query
def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
# Select Query
def select(conn,selct_sql):
    cur = conn.cursor()
    try:
        cur.execute(selct_sql)
        names = list(map(lambda x: x[0], cur.description))
        rows = cur.fetchall()
    except Error as e:
        names = "NA"
        rows = []
    return [names,rows]

# Update Query
def update(conn,update_sql):
    cur = conn.cursor()
    try:
        cur.execute(update_sql)
        #names = list(map(lambda x: x[0], cur.description))
        #rows = cur.fetchall()
        conn.commit()
    except Error as e:
        print("Failed to update")

# Delete Query
def delete(conn,delete_sql):
    cur = conn.cursor()
    try:
        cur.execute(delete_sql)
    except Error as e:
        print("not deleted")
        
# Create View Query
def create_view(conn, create_view_sql):
    try:
        c = conn.cursor()
        c.execute(create_view_sql)
    except Error as e:
        print(e)

Converting csv file to SQLite file

In [4]:
def save_as_sql(file,conn,table_name,index=False):
    read_csv = pd.read_csv(file)
    read_csv.to_sql(table_name, conn, index)

Saving tables

In [5]:
conn = create_connection('weather_data',True)
save_as_sql('city_attributes.csv',conn,'cities',False)
save_as_sql('humidity.csv',conn,'humidity',False)
save_as_sql('pressure.csv',conn,'pressure',False)
save_as_sql('temperature.csv',conn,'temperature',False)
save_as_sql('weather_description.csv',conn,'weather_description',False)
save_as_sql('wind_direction.csv',conn,'wind_direction',False)
save_as_sql('wind_speed.csv',conn,'wind_speed',False)
save_as_sql('USA_Historical_Tornado_Mine.csv',conn,'USA_Historical_Tornado',False)
save_as_sql('USA_CityCoordinates.csv',conn,'USA_City_Coordinates',False)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [ ]:
#save_as_sql('USA_Historical_Tornado_Mine.csv',conn,'USA_Historical_Tornado4',False)

In [ ]:
#update(conn,"Update USA_Historical_Tornado4 set USA_Historical_Tornado3.date = strftime('%YYYY-%mm-%dd', date) where Yr = Substr(date,7,10) and mo = Substr(date,4,5) and dy = Substr(date,1,2)")

In [6]:
[a,b] = select(conn,"Select * from USA_Historical_Tornado LIMIT 5")
print(b)

[(0, 1, 1950, 1, 3, '1950-01-03', '11:00:00', '11:00:00', 3, 'MO', 29, 1, 3, 3, 0, 6.0, 0.0, 38.77, -90.22, 38.83, -90.03, 9.5, 150, 2, 0, 1, 0, 0, 0, 0, 0), (1, 2, 1950, 1, 3, '1950-01-03', '11:55:00', '12:00:00', 3, 'IL', 17, 2, 3, 3, 0, 5.0, 0.0, 39.1, -89.3, 39.12, -89.23, 3.6, 130, 1, 1, 1, 135, 0, 0, 0, 0), (2, 3, 1950, 1, 3, '1950-01-03', '16:00:00', '16:00:00', 3, 'OH', 39, 1, 1, 1, 0, 4.0, 0.0, 40.88, -84.58, 0.0, 0.0, 0.1, 10, 1, 1, 1, 161, 0, 0, 0, 0), (3, 4, 1950, 1, 13, '1950-01-13', '5:25:00', '5:00:00', 3, 'AR', 5, 1, 3, 1, 1, 3.0, 0.0, 34.4, -94.37, 0.0, 0.0, 0.6, 17, 1, 1, 1, 113, 0, 0, 0, 0), (4, 5, 1950, 1, 25, '1950-01-25', '19:30:00', '20:00:00', 3, 'MO', 29, 2, 2, 5, 0, 5.0, 0.0, 37.6, -90.68, 37.63, -90.65, 2.3, 300, 1, 1, 1, 93, 0, 0, 0, 0)]


Selecting cities of USA

In [7]:
delete(conn,"DELETE FROM cities WHERE country != 'United States'")
delete(conn,"DELETE FROM cities WHERE City = 'San Francisco'")
delete(conn,"DELETE FROM cities WHERE City = 'Miami'")
delete(conn,"DELETE FROM cities WHERE City = 'New York'")
delete(conn,"Delete FROM humidity where datetime = '2012-10-01 12:00:00'")
delete(conn,"Delete FROM pressure where datetime = '2012-10-01 12:00:00'")
delete(conn,"Delete FROM wind_speed where datetime = '2012-10-01 12:00:00'")
delete(conn,"Delete FROM wind_direction where datetime = '2012-10-01 12:00:00'")
delete(conn,"Delete FROM temperature where datetime = '2012-10-01 12:00:00'")
"""
San Francisco
Miami
New York
"""

deleted
deleted
deleted
deleted
deleted
deleted
deleted
deleted
deleted


'\nSan Francisco\nMiami\nNew York\n'

In [ ]:
delete(conn,"drop TABLE USA_Historical_Tornado")
#delete(conn,"drop TABLE wind_direction")
#delete(conn,"drop TABLE wind_speed")
#delete(conn,"drop TABLE temperature")
#delete(conn,"drop TABLE pressure")

In [8]:
[names,cols] = select(conn,"select * from cities")
cities_list = []
for val in cols:
    cities_list.append(val[1])

In [9]:
for city in cities_list:
    humidity_nullvalues_update = """Update humidity set ["""+city+"""] = (SELECT avg_val 
                                from (Select AVG(h2.["""+city+"""]) as avg_val, Substr(h2.datetime,1,7) as tdatet FROM humidity as h2 group by Substr(h2.datetime,1,7)) as temp1 
                                where Substr(humidity.datetime,1,7)=temp1.tdatet)
                                where datetime not in (SELECT h1.datetime FROM humidity as h1 where  ["""+city+"""] >= 0)"""
    update(conn,humidity_nullvalues_update)

    temperature_nullvalues_update = """Update temperature set ["""+city+"""] = (SELECT avg_val 
                                       from (Select AVG(t2.["""+city+"""]) as avg_val, Substr(t2.datetime,1,7) as tdatet FROM temperature as t2 group by Substr(t2.datetime,1,7)) as temp1 
                                       where Substr(temperature.datetime,1,7)=temp1.tdatet)
    where datetime not in (SELECT t2.datetime FROM temperature as t2 where  ["""+city+"""] >= 0)"""
    update(conn,temperature_nullvalues_update)
    
    pressure_nullvalues_update = """Update pressure set ["""+city+"""] = (SELECT avg_val 
                                       from (Select AVG(p2.["""+city+"""]) as avg_val, Substr(p2.datetime,1,7) as tdatet FROM pressure as p2 group by Substr(p2.datetime,1,7)) as temp1 
                                       where Substr(pressure.datetime,1,7)=temp1.tdatet)
    where datetime not in (SELECT p2.datetime FROM pressure as p2 where  ["""+city+"""] >= 0)"""
    update(conn,pressure_nullvalues_update)
    
    winddir_nullvalues_update = """Update wind_direction set ["""+city+"""] = (SELECT avg_val 
                                       from (Select AVG(wd2.["""+city+"""]) as avg_val, Substr(wd2.datetime,1,7) as tdatet FROM wind_direction as wd2 group by Substr(wd2.datetime,1,7)) as temp1 
                                       where Substr(wind_direction.datetime,1,7)=temp1.tdatet)
    where datetime not in (SELECT wd2.datetime FROM wind_direction as wd2 where  ["""+city+"""] >= 0)"""
    update(conn,winddir_nullvalues_update)
    
    windsp_nullvalues_update = """Update wind_speed set ["""+city+"""] = (SELECT avg_val 
                                       from (Select AVG(ws2.["""+city+"""]) as avg_val, Substr(ws2.datetime,1,7) as tdatet FROM wind_speed as ws2 group by Substr(ws2.datetime,1,7)) as temp1 
                                       where Substr(wind_speed.datetime,1,7)=temp1.tdatet)
    where datetime not in (SELECT ws2.datetime FROM wind_speed as ws2 where  ["""+city+"""] >= 0)"""
    update(conn,windsp_nullvalues_update)

In [10]:
def coord_round_dwn(a):
    if(a<0):
        b = math.floor(abs(a));
        return (b*(-1));
    else:
        return (round(a))
    
def getcities(tl):
    att = {}
    for at in tl:
        if(at in att):
            att[at] = att[at]+1;
        else:
            att[at] = 1
    #print(att)
    return(list(att.keys()))

In [18]:
import math
str = '''Select U2.City,U2.Latitude,U2.Longitude,U1.state_name,U1.lat,U1.lng 
from USA_City_Coordinates as U1 INNER JOIN cities as U2 on U1.city = U2.City 
WHERE U2.Country='United States' and round(U1.lat) = round(U2.Latitude) and round(U1.lng) = round(U2.Longitude)'''
#wind_speed WHERE datetime like '10/1/2012%'";

[a,b] = select(conn,str)
lama = []
asas = []
for l1 in b:
    l = (l1[0],coord_round_dwn(l1[1]),coord_round_dwn(l1[2]),l1[3],coord_round_dwn(l1[4]),coord_round_dwn(l1[5]))
    lama.append(l)
    asas.append(l[0])
final_cities = getcities(asas)

In [19]:
state_abbreviations = {"Alabama":"AL","Alaska":"AK","Arizona":"AZ","Arkansas":"AR","California":"CA","Colorado":"CO","Connecticut":"CT","Delaware":"DE","Florida":"FL","Georgia":"GA","Hawaii":"HI","Idaho":"ID","Illinois":"IL","Indiana":"IN","Iowa":"IA","Kansas":"KS","Kentucky":"KY","Louisiana":"LA","Maine":"ME","Maryland":"MD","Massachusetts":"MA","Michigan":"MI","Minnesota":"MN","Mississippi":"MS","Missouri":"MO","Montana":"MT","Nebraska":"NE","Nevada":"NV","New Hampshire":"NH","New Jersey":"NJ","New Mexico":"NM","New York":"NY","North Carolina":"NC","North Dakota":"ND","Ohio":"OH","Oklahoma":"OK","Oregon":"OR","Pennsylvania":"PA","Rhode Island":"RI","South Carolina":"SC","South Dakota":"SD","Tennessee":"TN","Texas":"TX","Utah":"UT","Vermont":"VT","Virginia":"VA","Washington":"WA","West Virginia":"WV","Wisconsin":"WI","Wyoming":"WY"}
for state in state_abbreviations:
    query = "Update USA_Historical_Tornado set st = '"+state+"' where USA_Historical_Tornado.st = '"+state_abbreviations[state]+"'"
    update(conn,query)
    
    

In [150]:
str = """CREATE VIEW CityDetails (City,StateName,Latitude,Longitude)
             AS
             Select U1.City,U1.state_name,U2.Latitude,U2.Longitude
             from USA_City_Coordinates as U1 INNER JOIN cities as U2 on U1.city = U2.City
             WHERE U2.Country='United States' and round(U1.lat) = round(U2.Latitude) and round(U1.lng) = round(U2.Longitude)"""

(create_view(conn,str))

table CityDetails already exists


In [25]:
for city in final_cities:
    str1 = """CREATE VIEW ["""+city+"""_info] (Date,Time,Humidity,Weather_Description,Pressure,Temperature,Wind_Direction,Wind_Speed)
             AS 
             Select Substr(h1.datetime,1,10),Substr(h1.datetime,12,19),h1.["""+city+"""] as Humidity,wd1.["""+city+"""] as Weather_Description,
             p1.["""+city+"""] as Pressure,t1.["""+city+"""] as Temperature,widir.["""+city+"""] as Wind_Direction,
             wsp.["""+city+"""] as Wind_Speed
             from humidity as h1 
             INNER JOIN weather_description as wd1 on h1.datetime=wd1.datetime 
             INNER JOIN pressure as p1 on h1.datetime=p1.datetime 
             INNER JOIN temperature as t1 on h1.datetime=t1.datetime 
             INNER JOIN wind_direction as widir on h1.datetime=widir.datetime 
             INNER JOIN wind_speed as wsp on h1.datetime=wsp.datetime"""
             #WHERE h1.datetime like '2012-10-01%' and """;
    #print(str)
    create_view(conn,str1)
    #print(a)
    #print(b)
    #break;

table [Portland_info] already exists
table [Seattle_info] already exists
table [Los Angeles_info] already exists
table [San Diego_info] already exists
table [Las Vegas_info] already exists
table [Albuquerque_info] already exists
table [Denver_info] already exists
table [Dallas_info] already exists
table [Houston_info] already exists
table [Kansas City_info] already exists
table [Minneapolis_info] already exists
table [Chicago_info] already exists
table [Nashville_info] already exists
table [Indianapolis_info] already exists
table [Atlanta_info] already exists
table [Detroit_info] already exists
table [Jacksonville_info] already exists
table [Charlotte_info] already exists
table [Pittsburgh_info] already exists
table [Philadelphia_info] already exists
table [Boston_info] already exists


In [28]:
#index1 = int(input("Please enter index number for desired city values:"))
#print('\033[1m'+cityview_dict[index1]+" Data")
#city_val = cityview_list[index1]
city_val = 'Dallas_info'
str3 = "SELECT * FROM ["+city_val+"] where Date like '%2015-06-23%'";
city_df = pd.read_sql_query(str3,conn)
city_df

,Date,Time,Humidity,Weather_Description,Pressure,Temperature,Wind_Direction,Wind_Speed
0,2015-06-23,00:00:00,51.0,sky is clear,1012.0,305.538674,191.0,6.0
1,2015-06-23,01:00:00,53.0,sky is clear,1029.0,304.819667,188.0,6.0
2,2015-06-23,02:00:00,57.0,sky is clear,1029.0,303.445333,182.0,5.0
3,2015-06-23,03:00:00,60.0,sky is clear,1030.0,303.153000,189.0,5.0
4,2015-06-23,04:00:00,64.0,sky is clear,1030.0,301.418000,178.0,4.0
5,2015-06-23,05:00:00,67.0,sky is clear,1031.0,300.765000,181.0,4.0
6,2015-06-23,06:00:00,70.0,sky is clear,1031.0,300.112000,185.0,4.0
7,2015-06-23,07:00:00,73.0,sky is clear,1031.0,299.459000,187.0,4.0
8,2015-06-23,08:00:00,77.0,sky is clear,1031.0,298.806000,191.0,4.0
9,2015-06-23,09:00:00,80.0,sky is clear,1032.0,298.457500,195.0,3.0


In [61]:
select(conn,"Select * from CityDetails")

[['City', 'StateName', 'Latitude', 'Longitude'],
 [('Portland', 'Oregon', 45.523449, -122.676208),
  ('Seattle', 'Washington', 47.606209, -122.33206899999999),
  ('Los Angeles', 'California', 34.052231, -118.243683),
  ('San Diego', 'California', 32.715328, -117.157257),
  ('Las Vegas', 'Nevada', 36.174969, -115.137222),
  ('Albuquerque', 'New Mexico', 35.084492, -106.65113799999999),
  ('Denver', 'Colorado', 39.739151, -104.984703),
  ('Dallas', 'Texas', 32.783058000000004, -96.806671),
  ('Houston', 'Texas', 29.763281, -95.363274),
  ('Kansas City', 'Kansas', 39.099731, -94.578568),
  ('Kansas City', 'Missouri', 39.099731, -94.578568),
  ('Minneapolis', 'Minnesota', 44.979969, -93.26384),
  ('Chicago', 'Illinois', 41.850029, -87.650047),
  ('Nashville', 'Tennessee', 36.165890000000005, -86.784439),
  ('Indianapolis', 'Indiana', 39.768378999999996, -86.158043),
  ('Atlanta', 'Georgia', 33.749001, -84.387978),
  ('Detroit', 'Michigan', 42.331429, -83.045753),
  ('Jacksonville', 'Florid

In [103]:
#df1 = pd.read_sql_query("""Select UHT.date,UHT.Time,UHT.len,UHT.[start lat],UHT.[start lon],UHT.[fin lat],UHT.[fin lon]
#from USA_Historical_Tornado as UHT INNER JOIN CityDetails as CD on UHT.st = CD.StateName 
#WHERE CD.Latitude>=[start lat] and CD.Latitude<=[fin lat]""",conn)

#df1
asa = '[Portland_More_Info]','[Seattle_More_Info]','[Los Angeles_More_Info]','[San Diego_More_Info]','[Las Vegas_More_Info]','[Albuquerque_More_Info]','[Denver_More_Info]','[Dallas_More_Info]','[Houston_More_Info]','[Kansas City_More_Info]','[Minneapolis_More_Info]','[Chicago_More_Info]','[Nashville_More_Info]','[Indianapolis_More_Info]','[Atlanta_More_Info]','[Detroit_More_Info]','[Jacksonville_More_Info]','[Charlotte_More_Info]','[Pittsburgh_More_Info]','[Philadelphia_More_Info]','[Boston_More_Info]'
for a in asa:
    delete(conn,"drop View "+a)

for city in final_cities:
    str1 = """CREATE VIEW ["""+city+"""_More_Info] (Date,Time,City,State,Len,Start_lat,Start_Lon,Final_Lat,Final_Long)
             AS 
             Select UHT.date,UHT.Time,CD.City,CD.StateName,UHT.len,UHT.[start lat],UHT.[start lon],UHT.[fin lat],UHT.[fin lon]
             from USA_Historical_Tornado as UHT INNER JOIN CityDetails as CD on CD.City = '"""+city+"""' 
             WHERE (CD.Latitude as INT)=cast([start lat] as INT) and cast(CD.Longitude as INT)=cast([fin lon] as INT)"""
             #WHERE h1.datetime like '2012-10-01%' and """;
    #print(str)

    create_view(conn,str1)

#Date','Time',  'Humidity',  'Weather_Description',  'Pressure',  'Temperature',  'Wind_Direction',  'Wind_Speed'
#Dallas_info

#df1[1:10]

In [143]:
dfs1 = pd.read_sql_query("Select * from Boston_More_Info where Date like '%201%'",conn)
dfs1

,Date,Time,City,State,Len,Start_lat,Start_Lon,Final_Lat,Final_Long
0,2011-06-01,14:00:00,Boston,Massachusetts,37.58,42.1000,-72.7500,42.1045,-71.9900
1,2013-05-09,15:00:00,Boston,Massachusetts,0.38,42.0981,-71.0962,42.1023,-71.0915
2,2014-07-28,8:00:00,Boston,Massachusetts,1.93,42.3980,-71.0219,42.4225,-71.0037
3,2014-08-31,18:00:00,Boston,Massachusetts,2.06,42.2400,-71.8000,42.2600,-71.7700
4,2015-06-23,16:00:00,Boston,Massachusetts,0.48,42.0650,-71.3335,42.0670,-71.3245
5,2015-06-23,17:00:00,Boston,Massachusetts,0.52,42.5119,-71.9041,42.5100,-71.8942


In [151]:
awqe = ['[Portland_Final_Info]','[Seattle_Final_Info]','[Los Angeles_Final_Info]','[San Diego_Final_Info]','[Las Vegas_Final_Info]','[Albuquerque_Final_Info]','[Denver_Final_Info]','[Dallas_Final_Info]','[Houston_Final_Info]','[Kansas City_Final_Info]','[Minneapolis_Final_Info]','[Chicago_Final_Info]','[Nashville_Final_Info]','[Indianapolis_Final_Info]','[Atlanta_Final_Info]','[Detroit_Final_Info]','[Jacksonville_Final_Info]','[Charlotte_Final_Info]','[Pittsburgh_Final_Info]','[Philadelphia_Final_Info]','[Boston_Final_Info]']
for a in awqe:
    delete(conn,"drop View "+a)

for city in final_cities:
    
    str1 = """CREATE VIEW ["""+city+"""_Final_Info] (Date,State,Humidity,Pressure,Temperature,Wind_Direction,Wind_Speed,Len,Start_lat,Start_Lon,Final_Lat,Final_Long )
             AS 
             Select DMI.Date,DMI.State,DI.Humidity,DI.Pressure,DI.Temperature,DI.Wind_Direction,DI.Wind_Speed,DMI.Len,DMI.Start_lat,DMI.Start_Lon,DMI.Final_Lat,DMI.Final_Long 
             from ["""+city+"""_More_info] as DMI INNER JOIN ["""+city+"""_info] as DI on DMI.date = DI.Date 
             WHERE DMI.Date = DI.Date"""
             #WHERE h1.datetime like '2012-10-01%' and """;
    #print(str)

    create_view(conn,str1)
    
#    df1 = pd.read_sql_query("""Select DMI.Date,DMI.Time,DI.Time,DMI.State,DI.Humidity,DI.Weather_Description,DI.Pressure,DI.Temperature,
#    DI.Wind_Direction,DI.Wind_Speed,DMI.Len,DMI.Start_lat,DMI.Start_Lon,DMI.Final_Lat,DMI.Final_Long 
#    from Dalls_More_info as DMI INNER JOIN Dallas_info as DI on DMI.date = DI.Date 
#    WHERE DMI.Date = DI.Date and DMI.Time = DI.Time""",conn)

#df1


In [152]:
#delete(conn,"drop View [Boston_Final_Info]")
df1 = pd.read_sql_query("Select * from [Dallas_Final_Info]",conn)
df1

,Date,State,Humidity,Pressure,Temperature,Wind_Direction,Wind_Speed,Len,Start_lat,Start_Lon,Final_Lat,Final_Long
0,2013-05-15,Texas,22.0,1020.0,302.530000,180.0,4.0,6.17,32.3250,-96.6310,32.3640,-96.5360
1,2013-05-15,Texas,29.0,1019.0,301.390000,117.0,1.0,6.17,32.3250,-96.6310,32.3640,-96.5360
2,2013-05-15,Texas,34.0,1020.0,299.510000,143.0,0.0,6.17,32.3250,-96.6310,32.3640,-96.5360
3,2013-05-15,Texas,37.0,1020.0,298.000000,250.0,0.0,6.17,32.3250,-96.6310,32.3640,-96.5360
4,2013-05-15,Texas,33.0,1021.0,296.770000,190.0,4.0,6.17,32.3250,-96.6310,32.3640,-96.5360
...,...,...,...,...,...,...,...,...,...,...,...,...
307,2015-12-26,Texas,83.0,1005.0,297.240000,160.0,9.0,13.04,32.7960,-96.5894,32.9669,-96.4950
308,2015-12-26,Texas,83.0,1005.0,297.157178,160.0,9.0,1.58,32.3340,-97.0120,32.3510,-96.9940
309,2015-12-26,Texas,83.0,1005.0,297.157178,160.0,9.0,8.52,32.4623,-96.9090,32.5699,-96.8376
310,2015-12-26,Texas,83.0,1005.0,297.157178,160.0,9.0,5.08,32.3855,-96.5300,32.4566,-96.5080


In [66]:
df1 = pd.read_sql_query("Select * from Dalls_More_info",conn)
#df1['date'] = pd.to_datetime(df1['date'])
#df2 = pd.read_sql_query("Select Date,Humidity,Weather_Description,Pressure,Temperature,Wind_Direction,Wind_Speed from Dallas_info",conn)
#df2['Date'] = pd.to_datetime(df2['Date'])
#df2
df1

,Date,Time,City,State,Len,Start_lat,Start_Lon,Final_Lat,Final_Long
0,1956-05-25,12:00:00,Albuquerque,New Mexico,15.80,35.0000,-106.3000,35.1800,-106.1300
1,2004-08-14,20:00:00,Albuquerque,New Mexico,0.10,34.1500,-103.2700,35.1500,-103.2700
2,1957-05-20,11:00:00,Denver,Colorado,70.10,39.2500,-102.4000,39.9000,-101.7800
3,1975-05-27,14:00:00,Denver,Colorado,12.60,39.6200,-102.4200,39.7700,-102.2800
4,1998-05-24,14:00:00,Denver,Colorado,2.00,39.7000,-104.5800,39.8200,-104.6000
...,...,...,...,...,...,...,...,...,...
182,2003-06-12,16:00:00,Pittsburgh,Pennsylvania,5.50,40.4200,-80.0700,40.4500,-80.0300
183,2011-04-28,1:00:00,Pittsburgh,Pennsylvania,8.46,40.3800,-77.5896,40.4669,-77.4763
184,1958-07-14,13:00:00,Philadelphia,Pennsylvania,9.80,39.9300,-75.1300,39.9700,-74.9300
185,2012-10-19,18:00:00,Philadelphia,Pennsylvania,16.56,39.8002,-76.2631,40.0112,-76.1151


In [31]:
#df1 = df1.assign(key=1)
#df2 = df2.assign(key=2)
#df10 = pd.merge(df1,df2, left_on="date",right_on="Date",how = "")
#df10 = pd.merge(df1,df2, on="key").drop('key',axis = 1)
df10 = pd.merge(df2,df1, left_on="Date",right_on="date",how = "left")
#df10 = pd.merge(df1,df2, on=['date','Date'],how='l')
df11 = df10
df11 = df11.dropna()
df11

,Date,Humidity,Weather_Description,Pressure,Temperature,Wind_Direction,Wind_Speed,date,st,len,start lat,start lon,fin lat,fin lon
0,2012-10-01,87.0,mist,1011.0,289.740000,340.0,3.0,2012-10-01,Alabama,0.52,32.8089,-86.2405,32.8157,-86.2367
1,2012-10-01,87.0,mist,1011.0,289.740000,340.0,3.0,2012-10-01,Tennessee,1.51,35.4900,-86.4900,35.5067,-86.4725
2,2012-10-01,87.0,mist,1011.0,289.740000,340.0,3.0,2012-10-01,North Carolina,0.31,34.3277,-78.5773,34.3317,-78.5748
3,2012-10-01,87.0,mist,1011.0,289.740000,340.0,3.0,2012-10-01,North Carolina,0.14,34.4111,-78.4724,34.4125,-78.4707
4,2012-10-01,87.0,mist,1011.0,289.740000,340.0,3.0,2012-10-01,Tennessee,1.96,36.0070,-86.1394,36.0342,-86.1291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90299,2015-12-28,74.0,heavy intensity rain,1014.0,275.510432,300.0,5.0,2015-12-28,Louisiana,0.60,30.0775,-90.5378,30.0826,-90.5296
90300,2015-12-28,74.0,heavy intensity rain,1014.0,275.510432,300.0,5.0,2015-12-28,Arkansas,5.75,34.7020,-90.9161,34.7843,-90.9013
90301,2015-12-28,74.0,heavy intensity rain,1014.0,275.510432,300.0,5.0,2015-12-28,Mississippi,5.59,31.5405,-89.5301,31.6138,-89.4898
90302,2015-12-28,74.0,heavy intensity rain,1014.0,275.510432,300.0,5.0,2015-12-28,Florida,0.78,30.7626,-87.2375,30.7736,-87.2348


In [ ]:
select(conn,"Select date,time,st,len from USA_Historical_Tornado")

In [ ]:
"""
slongs = [-71.13,-72.75,-72.48,-72.23,-72.0677,-70.5691,-71.0962,-72.3534,-73.1858,-71.0219,-71.8,-71.3335,-71.9041]
flongs = [-70.97,-71.99,-72.4,-72.2,-72.04,-70.5693,-71.0915,-72.3524,-73.1835,-71.0037,-71.77,-71.3245,-71.8942]

for i in range(0,len(slongs)):
    if(slongs[i] <= -71.0846 <= flongs[i]):
        print("A")
    else:
        print("B")
"""

In [ ]:
[a,b] = (select(conn,"Select * from USA_Historical_Tornado LIMIT 10"))
print(a)
print(b)